In [ ]:
from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [ ]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [ ]:
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [ ]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [ ]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [ ]:
from collections import Counter
from scipy.sparse import csr_matrix
import math

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

In [ ]:
def textrank_keysentence(sents, tokenize, min_count, similarity, df=0.85, max_iter=30, topk=5, min_sim=0.5):
    g = sent_graph(sents, tokenize, min_count, min_sim, similarity)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    sent = sent.replace('\n','').replace('\t','').replace('\r','')
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [ ]:
keyword_extractor = textrank_keyword(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)

keywords = keyword_extractor.summarize(sents, topk=30)

In [ ]:
summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.5)
keysents = summarizer.summarize(sents, topk=10)

In [ ]:
sent = text = "오늘은 발표회 날이다. 긴장이 되었다. 아침에 피자을 먹어서 기분이 좋아졌다! 발표회가 시작되고 1~2학년이 합쳐서 오카리나 연주을 들었다. 너무 아름다웠다. 마치 병아리가 아장아장 걸어다니는거 같다.그리고 인상깊은게 마지막에 오징어 게임ost가 인상이 깊다. 왜냐하면 1~2학년이 직접 녹음하고 무궁화 꽃이 피었습니다라고 말하고 1~2학년 탈락!이라고 하고 그리고 오징어 게임의 한장면을 연기한게 인상이 깊었다.단체로 쓰러지는게 쉽지는 않을텐데 1~2학년의 협동심이 대단한거 같다. 피아노 바와후 애들은 피아노을 발표하는데 하나하나 너무 아름다웠다. 마치 하나의 벛꽃같았다. 특히 울면안돼 플라워 댄스가 너무 인상깊었다. 3~4학년 아쉽게도 나한텐 인상이 안깊다. 노력은 했지만 좀 아쉽다. 근데 5학년는 내가 진짜 칭찬하고 싶은게 노력을 많이한게 보인다. 왜냐하면 아침시간, 점심시간, 중간놀이 등 연습을 했다. 5학년의 리코더 발표는 마치 참 참새의 소리가 같다. 그리고 댄스배틀이 나한테 가장인상이 깊었다. 왜냐하면 그냥 흥이 나고 재밌고애들이 웃어줘서 힘이나  춤추다가 안경 떨어지고 넘어지고 목도 다쳤지만 난 뿌듯하다. 그리고 6학년은 대단하다.  우리반이 너무 자랑스럽다. 왜냐하면 리코더을 너무 잘하고 실러폰도 잘했다. 노래가 마치 산비둘기 단체 구구구구 하는거 같이 잘 맞춰가고 흥이 났다. 우리가 발표한 연주는 똥 밟았네, 할아버지 시계, 젓가락 행진곡을 연주 했다. 나는 뿌듯하면서 행복했다. 이런 친구들이 있어서 너무 든든했다. 이런 친구들이 있었기에 내가 연주을 잘할수 있었던 것이다. 마지막에 합창을 했는데 마치 밤하늘의 별 같았다. 너무 좋고 아름답고 음도 좋다. 합창을 하는데 팔로 하트하는게 뿌끄러웠지만 용기내서 했다. 그치만 하고 이발표회는 끝이 났다. 내 자신이 자랑스럽고 뿌듯한 날이고 이번 오늘 아니 이날은 Good 아니 Great 아니 Nice 아니 이 날은 가치로 매길수 없는 날이다"

In [ ]:
sent = sent.replace('\n','').replace('\t','').replace('\r','')

In [ ]:
sent

In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()
def kkma_tokenize(sent):
    #sent = sent.replace('\n','').replace('\t','').replace('\r','')
    words = kkma.pos(sent, join=True)
    print(words)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [ ]:
def subword_tokenizer(sent, n=3):
    def subword(token, n):
        if len(token) <= n:
            return [token]
        return [token[i:i+n] for i in range(len(token) - n)]
    return [sub for token in sent.split() for sub in subword(token, n)]

In [ ]:
textrank_keysentence(sent, subword_tokenizer, min_count=2, similarity=textrank_sent_sim)

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
summarize(sent, ratio=0.05)